In [1]:
from options.options import Options
import os
import torch
from build_dataset_model import build_loaders, build_model
from utils import get_model_attr, calculate_model_losses, tensor_aug
from collections import defaultdict
import math

In [2]:
args = Options().parse()
if (args.output_dir is not None) and (not os.path.isdir(args.output_dir)):
    os.mkdir(args.output_dir)
if (args.test_dir is not None) and (not os.path.isdir(args.test_dir)):
    os.mkdir(args.test_dir)

| options
dataset: suncg
suncg_train_dir: metadata/data_rot_train.json
suncg_val_dir: metadata/data_rot_val.json
suncg_data_dir: /home/yizhou/Research/SUNCG/suncg_data
loader_num_workers: 8
embedding_dim: 64
gconv_mode: feedforward
gconv_dim: 128
gconv_hidden_dim: 512
gconv_num_layers: 5
mlp_normalization: batch
vec_noise_dim: 0
layout_noise_dim: 32
batch_size: 16
num_iterations: 20000
eval_mode_after: -1
learning_rate: 7e-05
print_every: 10
checkpoint_every: 1000
snapshot_every: 10000
output_dir: ./checkpoints
checkpoint_name: latest_checkpoint
timing: False
multigpu: False
restore_from_checkpoint: False
checkpoint_start_from: None
test_dir: ./layouts_out
gpu_id: 0
KL_loss_weight: 0.1
use_AE: False
decoder_cat: True
train_3d: True
KL_linear_decay: False
use_attr_30: True
manual_seed: 42
batch_gen: False
measure_acc_l1_std: False
heat_map: False
draw_2d: False
draw_3d: False
fine_tune: False
gan_shade: False
blender_path: /home/yizhou/blender-2.92.0-linux64/blender
entropy_alpha: 0.1
r

In [3]:
args.suncg_train_dir = '/home/yizhou/Research/3D-FRONT-ToolBox/metadata43DSLN/all_room_info.json'

args.valid_types_dir = "/home/yizhou/Research/3D-FRONT-ToolBox/metadata43DSLN/valid_types.json"

args.suncg_val_dir = '/home/yizhou/Research/3D-FRONT-ToolBox/metadata43DSLN/all_room_info.json'

In [5]:
vocab, train_loader, val_loader = build_loaders(args)

Starting to read the json file for SUNCG
Training dataset has 62 scenes and 302 objects
(4.87 objects per image)
Starting to read the json file for SUNCG


In [6]:
dt = train_loader.dataset

In [7]:
dt[0]

/home/yizhou/Research/3D_SLN/data/suncg_dataset.py:298: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /opt/conda/conda-bld/pytorch_1607370172916/work/torch/csrc/utils/python_arg_parser.cpp:882.)
  real_objs = (objs != __room__).nonzero().squeeze(1)


(0,
 tensor([ 6, 11,  8, 13, 11,  1,  0]),
 tensor([[ 0.0749,  0.0000,  0.1435,  0.1642,  0.1761,  0.2528],
         [ 0.1721,  0.0000,  0.2464,  0.1978,  0.0393,  0.3012],
         [ 0.1042,  0.0000,  0.0546,  0.1292,  0.0414,  0.0948],
         [-0.0340,  0.0000,  0.1011,  0.0657,  0.0520,  0.3100],
         [ 0.0399,  0.0000,  0.2445,  0.0655,  0.0393,  0.2993],
         [ 0.0935,  0.5091,  0.0706,  0.1406,  0.2814,  0.2085],
         [ 0.0000,  0.0000,  0.0000,  4.5442,  2.5000,  3.3831]]),
 tensor([[4, 1, 0],
         [5, 3, 1],
         [4, 4, 2],
         [1, 2, 3],
         [1, 2, 4],
         [0, 9, 5]]),
 tensor([0, 0, 0, 6, 0, 0, 0]),
 tensor([2, 0, 0, 0, 0, 4, 0]))

In [9]:
model, model_kwargs = build_model(args, vocab)

In [10]:
vocab

{'object_idx_to_name': ['__room__',
  'lamp',
  'armoire',
  'chair',
  'table',
  'desk',
  'bed',
  'sofa',
  'stool',
  'shelf',
  'stand',
  'nightstand',
  'cabinet',
  'wardrobe',
  'frame',
  'armchair'],
 'object_name_to_idx': {'__room__': 0,
  'lamp': 1,
  'armoire': 2,
  'chair': 3,
  'table': 4,
  'desk': 5,
  'bed': 6,
  'sofa': 7,
  'stool': 8,
  'shelf': 9,
  'stand': 10,
  'nightstand': 11,
  'cabinet': 12,
  'wardrobe': 13,
  'frame': 14,
  'armchair': 15},
 'pred_idx_to_name': ['__in_room__',
  'left of',
  'right of',
  'behind',
  'in front of',
  'inside',
  'surrounding',
  'left touching',
  'right touching',
  'front touching',
  'behind touching',
  'front left',
  'front right',
  'back left',
  'back right',
  'on'],
 'pred_name_to_idx': {'__in_room__': 0,
  'left of': 1,
  'right of': 2,
  'behind': 3,
  'in front of': 4,
  'inside': 5,
  'surrounding': 6,
  'left touching': 7,
  'right touching': 8,
  'front touching': 9,
  'behind touching': 10,
  'front le

In [11]:
args.train_3d

True

In [12]:
print(model)
model.float().cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=args.learning_rate)

Sg2ScVAEModel(
  (obj_embeddings_ec): Embedding(17, 48)
  (pred_embeddings_ec): Embedding(16, 128)
  (obj_embeddings_dc): Embedding(17, 48)
  (pred_embeddings_dc): Embedding(16, 128)
  (attr_embedding_ec): Embedding(5, 16)
  (attr_embedding_dc): Embedding(5, 16)
  (box_embeddings): Linear(in_features=6, out_features=48, bias=True)
  (angle_embeddings): Embedding(24, 16)
  (box_mean_var): Sequential(
    (0): Linear(in_features=128, out_features=256, bias=True)
    (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=256, out_features=128, bias=True)
    (4): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
  )
  (box_mean): Sequential(
    (0): Linear(in_features=128, out_features=48, bias=True)
  )
  (box_var): Sequential(
    (0): Linear(in_features=128, out_features=48, bias=True)
  )
  (angle_mean_var): Sequential(
    (0): Linear(in_features=128, out_feature

In [13]:
for batch in train_loader:
    ids, objs, boxes, triples, angles, attributes, obj_to_img, triple_to_img = tensor_aug(batch)
    break

In [14]:
ids

tensor([12, 38, 46,  8, 23,  5, 31, 61, 58, 13,  6, 11,  2, 21,  1,  4],
       device='cuda:0')

In [15]:
angles

tensor([ 6,  0,  6,  0,  3,  0,  0, 18,  0,  0,  0,  6,  0,  0, 21,  6,  0, 18,
        18, 18,  0,  0, 18,  6, 18,  0,  0,  6,  0,  0,  0,  0, 23,  0,  0,  0,
         0,  0, 18,  0,  0,  6,  6,  6, 19, 18,  5,  0,  0,  0,  0,  6,  6,  6,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  6,  0,  0,  6,  0,  0, 18,
         0,  0,  0,  0,  0, 18,  0,  6, 18, 18, 18,  0, 18,  0,  0, 18, 18,  0,
        18,  0,  0, 18, 18, 18,  0,  0], device='cuda:0')

In [17]:
model_out = model(objs, triples, boxes, angles, attributes, obj_to_img)
mu, logvar, boxes_pred, angles_pred = model_out

In [18]:
model.decoder_cat

True

In [19]:
mu.shape

torch.Size([98, 64])

In [20]:
objs.shape

torch.Size([98])

In [21]:
model.pred_embeddings_ec

Embedding(16, 128)

In [22]:
torch.__version__

'1.7.1'

In [23]:
torch.version.cuda

'11.0'